In [3]:
import torch
from torch_geometric.datasets import PCPNetDataset
from torch_geometric.transforms import ToSparseTensor, KNNGraph, Compose
from normal_diffusion.data.patches import PatchDataloader
from normal_diffusion.data.transforms import DistanceToEdgeWeight, KeepNormals
from torch_geometric.loader import DataLoader

device = "cuda" if torch.cuda.is_available() else "cpu"
# Choose the root directory where you want to save the dataset
root = "../data/PCPNetDataset"
dataset = PCPNetDataset(
    root=root,
    category="NoNoise",
    split="train",
    transform=Compose([KeepNormals(), KNNGraph(k=6), DistanceToEdgeWeight(), ToSparseTensor()]),
)
# dataloader = PatchDataloader(dataset, batch_size=256, hops=15, transform=Compose([DistanceToEdgeWeight(), ToSparseTensor()]), limit_num_batches=1000) # can add ToSparseTensor conversion here 
# dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# dataloader = Batch.from_data_list(dataset)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)
print(len(dataloader))
first_collection = next(iter(dataloader))
first_collection = first_collection.to(device)
print(first_collection.x.shape)
print(first_collection.adj_t)
print(first_collection)

1
torch.Size([800000, 3])
SparseTensor(row=tensor([     0,      0,      0,  ..., 799999, 799999, 799999], device='cuda:0'),
             col=tensor([  2905,  22229,  41272,  ..., 751001, 782571, 799218], device='cuda:0'),
             val=tensor([0.1660, 0.1672, 0.1683,  ..., 0.1664, 0.1682, 0.1650], device='cuda:0'),
             size=(800000, 800000), nnz=4800000, density=0.00%)
DataBatch(x=[800000, 3], pos=[800000, 3], test_idx=[40000], adj_t=[800000, 800000, nnz=4800000], batch=[800000], ptr=[9])


In [4]:
import torch
from normal_diffusion.models import GCNModel
model = GCNModel().to(device)
t = torch.ones(first_collection.x.shape[0]).to(device)
predicted_normals = model(graph_data=first_collection, t=t)
print(predicted_normals)


tensor([[-1.4871, -0.8573, -0.5428],
        [-1.2113, -0.8346, -0.2968],
        [-1.1594, -0.7880, -0.3237],
        ...,
        [-1.2974, -3.5972, -1.1777],
        [-1.8986, -3.8313, -1.3911],
        [-1.5752, -4.1492, -1.0716]], device='cuda:0', grad_fn=<AddBackward0>)


In [5]:
import datetime
from diffusers import DDPMScheduler
from normal_diffusion.training.training import train_diffusion
from torch.utils.tensorboard import SummaryWriter
scheduler = DDPMScheduler(num_train_timesteps=5, beta_schedule="squaredcos_cap_v2", clip_sample=False)
# Setup TensorBoard
log_dir = "logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

writer = SummaryWriter(log_dir=log_dir)

train_diffusion(model=model, dataloader=dataloader, scheduler=scheduler, n_epochs=100, lr=1e-3, writer=writer, device=device)


/home/nirg/anaconda3/envs/normal-prediction/lib/python3.11/site-packages/diffusers/configuration_utils.py:134: FutureWarning: Accessing config attribute `num_train_timesteps` directly via 'DDPMScheduler' object attribute is deprecated. Please access 'num_train_timesteps' over 'DDPMScheduler's config object instead, e.g. 'scheduler.config.num_train_timesteps'.
  deprecate("direct config name access", "1.0.0", deprecation_message, standard_warn=False)


Epoch 1/100, Loss: -0.0150
Epoch 2/100, Loss: -0.0309
Epoch 3/100, Loss: -0.0461
Epoch 4/100, Loss: -0.0610
Epoch 5/100, Loss: -0.0757
Epoch 6/100, Loss: -0.0907
Epoch 7/100, Loss: -0.1062
Epoch 8/100, Loss: -0.1226
Epoch 9/100, Loss: -0.1405
Epoch 10/100, Loss: -0.1606
Epoch 11/100, Loss: -0.1830
Epoch 12/100, Loss: -0.2090
Epoch 13/100, Loss: -0.2364
Epoch 14/100, Loss: -0.2650
Epoch 15/100, Loss: -0.2876
Epoch 16/100, Loss: -0.3035
Epoch 17/100, Loss: -0.3175
Epoch 18/100, Loss: -0.3317
Epoch 19/100, Loss: -0.3444
Epoch 20/100, Loss: -0.3501
Epoch 21/100, Loss: -0.3570
Epoch 22/100, Loss: -0.3666
Epoch 23/100, Loss: -0.3744
Epoch 24/100, Loss: -0.3793
Epoch 25/100, Loss: -0.3825
Epoch 26/100, Loss: -0.3869
Epoch 27/100, Loss: -0.3913
Epoch 28/100, Loss: -0.3964
Epoch 29/100, Loss: -0.4018
Epoch 30/100, Loss: -0.4063
Epoch 31/100, Loss: -0.4109
Epoch 32/100, Loss: -0.4162
Epoch 33/100, Loss: -0.4201
Epoch 34/100, Loss: -0.4251
Epoch 35/100, Loss: -0.4299
Epoch 36/100, Loss: -0.4325
E